# 기업 성장 예측 모델링 - Feature 그룹별 분석
## 주요 수정 사항
1. ✅ **4가지 모델 명확히 정의**: LogReg, RF, XGB, LGBM
2. ✅ **하이퍼파라미터 튜닝 추가**: GridSearchCV 사용
3. ✅ **MLflow 로깅 구현**: 모든 실험 자동 기록
4. ✅ **5가지 평가지표 완비**: roc_auc, top20_precision, top50_precision, f1, brier_score
5. 🆕 **Feature 그룹별 실험**: 재무/특허 데이터 조합 비교

In [ ]:
# 기본 라이브러리
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# 모델링 라이브러리
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# 튜닝 및 검증
from sklearn.model_selection import GridSearchCV, GroupKFold
from sklearn.base import clone

# 평가 지표
from sklearn.metrics import (
    roc_auc_score,
    f1_score,
    brier_score_loss,
    precision_score
)

# MLflow 로깅
import mlflow
import mlflow.sklearn
from datetime import datetime

# tqdm 로딩
from tqdm.auto import tqdm

## 1. 데이터 로드

In [ ]:
# 데이터 경로 설정
DATA_DIR = Path("../modeling_data")

# 학습/테스트 데이터 로드
train_df = pd.read_csv(DATA_DIR / "train_dataset.csv")
test_df = pd.read_csv(DATA_DIR / "test_dataset.csv")

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")

# 데이터 확인
train_df.head()

## 2. 🆕 Feature 그룹 정의

In [ ]:
# =========================
# Feature 그룹 정의
# =========================

ID_COLS = ['기업명', '연도', 'target_year']
TARGET_COL = 'target_growth'

# 1. 재무 데이터
FIN_COLS = [
    'revenue_t1_scaled',
    'cagr_2y_scaled',
    'growth_recent_scaled',
    'growth_acceleration_scaled',
    'growth_volatility_scaled',
    'operating_margin_scaled',
    'capex_intensity_scaled',
    'capex_trend_scaled',
    'capex_vs_industry_scaled',
    'debt_ratio_scaled',
    'rnd_intensity_scaled',
    'profitable_years'
]

# 2. 특허 수치 데이터
PATENT_NUM_COLS = [
    'patent_count_5y',
    'patent_count_recent',
    'ipc_diversity',
    'patent_register_rate',
    'patent_citation_total',
    'patent_citation_avg',
    'patent_citation_age_adj_total',
    'patent_citation_age_adj_avg',
    'no_patent_flag'
]

# 3. 특허 임베딩 데이터
PATENT_EMB_COLS = [col for col in train_df.columns if col.startswith('patent_emb_')]

print("✅ Feature 그룹 정의 완료:")
print(f"  - 재무 데이터: {len(FIN_COLS)}개")
print(f"  - 특허 수치: {len(PATENT_NUM_COLS)}개")
print(f"  - 특허 임베딩: {len(PATENT_EMB_COLS)}개")
print(f"  - 전체: {len(FIN_COLS) + len(PATENT_NUM_COLS) + len(PATENT_EMB_COLS)}개")

In [ ]:
# =========================
# 실험할 Feature 조합 정의
# =========================

FEATURE_COMBINATIONS = {
    'FIN_ONLY': {
        'features': FIN_COLS,
        'description': '재무 데이터만'
    },
    'PATENT_NUM_ONLY': {
        'features': PATENT_NUM_COLS,
        'description': '특허 수치 데이터만'
    },
    # 'PATENT_EMB_ONLY': {
    #     'features': PATENT_EMB_COLS,
    #     'description': '특허 임베딩만'
    # },
    'FIN_PATENT_NUM': {
        'features': FIN_COLS + PATENT_NUM_COLS,
        'description': '재무 + 특허 수치'
    },
    # 'FIN_PATENT_EMB': {
    #     'features': FIN_COLS + PATENT_EMB_COLS,
    #     'description': '재무 + 특허 임베딩'
    # },
    # 'PATENT_NUM_EMB': {
    #     'features': PATENT_NUM_COLS + PATENT_EMB_COLS,
    #     'description': '특허 수치 + 임베딩'
    # },
    'ALL': {
        'features': FIN_COLS + PATENT_NUM_COLS + PATENT_EMB_COLS,
        'description': '전체 데이터'
    }
}

print("\n✅ 실험할 Feature 조합:")
for name, config in FEATURE_COMBINATIONS.items():
    print(f"  - {name:20s}: {config['description']:20s} ({len(config['features'])}개 feature)")

In [ ]:
# Target과 메타 데이터 분리
y_train = train_df[TARGET_COL]
y_test = test_df[TARGET_COL]
years_train = train_df['연도']
groups_train = train_df['기업명']

print(f"\nTarget distribution (train): {y_train.value_counts().to_dict()}")
print(f"Target distribution (test): {y_test.value_counts().to_dict()}")

## 3. 평가 지표 함수 정의

In [ ]:
def evaluate_model(y_true, y_pred, y_proba):
    """
    모델 평가 함수
    """
    # 1. ROC-AUC
    roc_auc = roc_auc_score(y_true, y_proba)
    
    # 2. F1 Score
    f1 = f1_score(y_true, y_pred)
    
    # 3. Brier Score
    brier = brier_score_loss(y_true, y_proba)
    
    # 4. Top-20 Precision
    top_20_idx = np.argsort(y_proba)[-20:]
    top_20_precision = y_true.iloc[top_20_idx].mean() if len(top_20_idx) > 0 else 0.0
    
    # 5. Top-50 Precision
    top_50_idx = np.argsort(y_proba)[-50:]
    top_50_precision = y_true.iloc[top_50_idx].mean() if len(top_50_idx) > 0 else 0.0
    
    return {
        'roc_auc': roc_auc,
        'f1': f1,
        'brier': brier,
        'top20_precision': top_20_precision,
        'top50_precision': top_50_precision
    }

## 4. 검증 방법 정의

In [ ]:
def holdout_validation(model, X, y, years):
    train_mask = years <= 2022
    val_mask = years == 2023

    X_tr = X.loc[train_mask]
    y_tr = y.loc[train_mask]
    X_val = X.loc[val_mask]
    y_val = y.loc[val_mask]

    if y_tr.nunique() < 2 or y_val.shape[0] == 0:
        return None

    model_clone = clone(model)
    model_clone.fit(X_tr, y_tr)

    y_pred = model_clone.predict(X_val)
    y_proba = model_clone.predict_proba(X_val)[:, 1]

    return evaluate_model(y_val, y_pred, y_proba)

In [ ]:
def group_kfold_validation(model, X, y, groups, n_splits=5):
    """GroupKFold Validation: 기업 단위로 fold 분할"""
    gkf = GroupKFold(n_splits=n_splits)
    scores = []

    for train_idx, val_idx in gkf.split(X, y, groups):
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model_clone = clone(model)
        model_clone.fit(X_tr, y_tr)
        
        y_pred = model_clone.predict(X_val)
        y_proba = model_clone.predict_proba(X_val)[:, 1]

        scores.append(evaluate_model(y_val, y_pred, y_proba))

    return pd.DataFrame(scores).mean().to_dict()

In [ ]:
def rolling_validation(model, X, y, years):
    """Rolling Window Validation: 시계열 순차 검증"""
    fold_scores = []

    for train_end in sorted(years.unique()):
        val_year = train_end + 1

        train_mask = years <= train_end
        val_mask = years == val_year

        X_tr = X.loc[train_mask]
        y_tr = y.loc[train_mask]
        X_val = X.loc[val_mask]
        y_val = y.loc[val_mask]

        if X_tr.shape[0] == 0 or X_val.shape[0] == 0:
            continue
        if y_tr.nunique() < 2:
            continue

        model_clone = clone(model)
        model_clone.fit(X_tr, y_tr)

        y_pred = model_clone.predict(X_val)
        y_proba = model_clone.predict_proba(X_val)[:, 1]

        fold_scores.append(evaluate_model(y_val, y_pred, y_proba))

    if len(fold_scores) == 0:
        return None

    return pd.DataFrame(fold_scores).mean().to_dict()

## 5. 모델 정의 및 하이퍼파라미터 그리드

In [ ]:
# 4가지 모델과 하이퍼파라미터 그리드 정의

models_and_params = {
    'LogReg': {
        'model': LogisticRegression(random_state=42, max_iter=1000),
        'params': {
            'C': [0.01, 0.1, 1, 10, 100],
            'penalty': ['l1', 'l2'],
            'solver': ['liblinear'],
            'class_weight': [None, 'balanced']
        }
    },
    
    'RF': {
        'model': RandomForestClassifier(random_state=42, n_jobs=-1),
        'params': {
            'n_estimators': [100, 200, 300],
            'max_depth': [10, 20, 30, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'class_weight': [None, 'balanced']
        }
    },
    
    'XGB': {
        'model': XGBClassifier(random_state=42, eval_metric='logloss', n_jobs=-1),
        'params': {
            'n_estimators': [100, 200, 300],
            'max_depth': [3, 5, 7, 10],
            'learning_rate': [0.01, 0.05, 0.1],
            'subsample': [0.8, 0.9, 1.0],
            'colsample_bytree': [0.8, 0.9, 1.0],
            'scale_pos_weight': [1, 2, 3]
        }
    },
    
    'LGBM': {
        'model': LGBMClassifier(random_state=42, verbose=-1, n_jobs=-1),
        'params': {
            'n_estimators': [100, 200, 300],
            'max_depth': [3, 5, 7, 10],
            'learning_rate': [0.01, 0.05, 0.1],
            'num_leaves': [31, 50, 70],
            'subsample': [0.8, 0.9, 1.0],
            'colsample_bytree': [0.8, 0.9, 1.0],
            'class_weight': [None, 'balanced']
        }
    }
}

print("✅ 정의된 모델:")
for name in models_and_params.keys():
    print(f"  - {name}")

## 6. MLflow 설정

In [ ]:
# MLflow 실험 설정
experiment_name = "Growth_Prediction_Feature_Groups"
mlflow.set_experiment(experiment_name)

print(f"✅ MLflow 실험 설정 완료: {experiment_name}")
print(f"   추적 URI: {mlflow.get_tracking_uri()}")

## 7. 🆕 Feature 그룹별 실험 실행

In [ ]:
# =========================================================
# 🔥 탐색용 경량 실험 코드
# - 목적: Feature Group × Model 빠른 스크리닝
# =========================================================

from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from sklearn.base import clone

fast_results = []

print("🚀 탐색용 경량 실험 시작")

for feature_group_name, feature_config in tqdm(
    FEATURE_COMBINATIONS.items(),
    desc="🧩 Feature Groups"
):
    feature_cols = feature_config["features"]
    X_train_group = train_df[feature_cols]

    for model_name, model_config in tqdm(
        models_and_params.items(),
        desc=f"🤖 Models ({feature_group_name})",
        leave=False
    ):
        model = clone(model_config["model"])

        # -----------------------------
        # 🔁 Rolling Validation만 수행
        # -----------------------------
        rolling_scores = rolling_validation(
            model,
            X_train_group,
            y_train,
            years_train
        )

        if rolling_scores is None:
            continue

        result_row = {
            "feature_group": feature_group_name,
            "n_features": len(feature_cols),
            "model": model_name,
            **rolling_scores
        }

        fast_results.append(result_row)

# 결과 DataFrame
fast_results_df = pd.DataFrame(fast_results)

print("\n✅ 탐색용 실험 완료")
fast_results_df.sort_values(
    by="top20_precision", ascending=False
).head(10)
